# BottomTrendFollow Researches



In [1]:
%matplotlib inline
import research_kit as rk

bot = rk.BottomTrendFollow(db_client=rk.mysql_client, exchange_client=rk.bitmex_exchange_client, is_backtest=True)

downloading BTC/USD data on bitmex
downloading 2019-12-28 19:36:30 ~ 2019-12-28 19:39:29.900881 data
100.0% completed


In [2]:
def show_bottom_trend_follow_params(bot):
    rk.pprint(bot.combined_params)
    
show_bottom_trend_follow_params(bot)

{'bot_name': 'bottom_trend_follow',
 'bottom_trend_tick': 12,
 'close_position_on_do_nothing': True,
 'inverse_trading': False,
 'middle_trend_tick': 6,
 'timeframe': 60,
 'top_trend_tick': 3}


In [3]:
def generate_transaction_log(bot, close_position_on_do_nothing=True, inverse_trading=False):
    #backtest_start_time = rk.datetime.now() - rk.timedelta(days=180)
    #backtest_end_time = rk.datetime.now()
    backtest_start_time = rk.datetime(2019,9,28,2,0,0)
    backtest_end_time = rk.datetime(2019,12,27,1,0,0)
    
    ohlcv_df_1min = rk.dataset_manager.get_ohlcv(start_time=backtest_start_time, end_time=backtest_end_time)
    print("dataset size:" + str(len(ohlcv_df_1min)))
    
    timeframe_params = [60,30,10,1]
    bottom_trend_tick_params = [5,10,15,20,25,30,35,40,45,50]
    middle_trend_tick_params = [3,5,7,9,12,15,18,21,24,27,30]
    top_trend_tick_params = [1,2,3,4,5,6,7,8]
    
    for timeframe in timeframe_params:
        print("timeframe=>" + str(timeframe))
        for bottom_trend_tick in bottom_trend_tick_params:
            bottom_loop_start = rk.datetime.now()
            for middle_trend_tick in middle_trend_tick_params:           
                for top_trend_tick in top_trend_tick_params:
                    
                    
                    if bottom_trend_tick <= middle_trend_tick or middle_trend_tick <= top_trend_tick:
                        continue
                        
                    default_params = {
                        "bot_name": bot.bot_name,
                        "close_position_on_do_nothing": close_position_on_do_nothing,
                        "inverse_trading": inverse_trading,
                        "timeframe": int(timeframe)
                    }
                    
                    specific_params = {
                        "bottom_trend_tick": int(bottom_trend_tick),
                        "middle_trend_tick": int(middle_trend_tick),
                        "top_trend_tick": int(top_trend_tick)
                    }
                    
                    bot.reset_backtest_result_with_params(default_params, specific_params)
                    before_run = rk.datetime.now()
                    bot.run(ohlcv_df=ohlcv_df_1min[::timeframe], backtest_start_time=backtest_start_time,
                           backtest_end_time=backtest_end_time)
                    print("bottom_trend_tick=>" + str(bottom_trend_tick) +\
                          " midle_trend_tick=>" + str(middle_trend_tick) +\
                          " top_trend_tick=>" + str(top_trend_tick) + " time:" + str(rk.datetime.now() - before_run))
                    
            print("=bottom trend tick:" + str(bottom_trend_tick) + " takes")
            bottom_loop_end = rk.datetime.now()
            print(bottom_loop_end - bottom_loop_start)
        bot.bulk_insert()
            
#generate_transaction_log(bot)
# it takes 4hours to calculate all wuth insert after getting transaction log before commit "optimize metrics calc"
generate_transaction_log(bot, inverse_trading=True)

Loading OHLCV data from bitmex_original_ohlcv_1min now...
Done
dataset size:129481
timeframe=>60
3234
711 transaction log added
最後の行
id
3227    3234
Name: backtest_summary_id, dtype: int64
     backtest_start_time backtest_end_time             bot_name  \
id                                                                
3234                None              None  bottom_trend_follow   

      initial_balance account_currency total_entry total_max_holding_ms  \
id                                                                        
3234            100.0              USD        None                 None   

     total_average_holding_ms total_min_holding_ms total_return  \
id                                                                
3234                     None                 None         None   

     total_return_average total_standard_deviation total_skewness  \
id                                                                  
3234                 None                  

bottom_trend_tick=>5 midle_trend_tick=>3 top_trend_tick=>1 time:0:00:01.201757
3235
605 transaction log added
最後の行
id
3228    3235
Name: backtest_summary_id, dtype: int64
     backtest_start_time backtest_end_time             bot_name  \
id                                                                
3235                None              None  bottom_trend_follow   

      initial_balance account_currency total_entry total_max_holding_ms  \
id                                                                        
3235            100.0              USD        None                 None   

     total_average_holding_ms total_min_holding_ms total_return  \
id                                                                
3235                     None                 None         None   

     total_return_average total_standard_deviation total_skewness  \
id                                                                  
3235                 None                     None           

AttributeError: 'NoneType' object has no attribute '__table__'

In [ ]:
lastid = int(rk.mysql_client.get_last_row("backtest_summary").index.array[0])
print(lastid)
from model.backtest_summary import BacktestSummary
print(rk.mysql_client.session.query(BacktestSummary).filter(BacktestSummary.id==3227).first())

In [ ]:
def get_joined_params_and_summary(bot):
    backtest_management = bot.backtest_management_table()
    backtest_summary = rk.BacktestSummary()
    
    query_management = "SELECT * FROM " + bot.backtest_management_table_name + ";"
    query_summary = "SELECT * FROM backtest_summary;"
    
    backtest_management_df = rk.mysql_client.exec_sql(query_management)
    backtest_summary_df = rk.mysql_client.exec_sql(query_summary)
    
    merged_backtest_results = rk.pd.merge(backtest_management_df, backtest_summary_df, left_on="backtest_summary_id", right_on="id")
    return merged_backtest_results
    
merged_results = get_joined_params_and_summary(bot)
merged_results.style.set_table_styles([dict(selector="th",props=[('max-width', '50px')])])
print(merged_results)
    

In [ ]:
def profit_factor_analysis(df):
    columns_picked_up = df.sort_values("profit_factor", ascending=False).loc[:,[
        "timeframe",
        "bottom_trend_tick",
        "middle_trend_tick",
        "top_trend_tick",
        "profit_factor"
    ]]
    
    timeframe_params = [60,30,10,1]
    
    fig_profit_factor_histogram = rk.plt.figure()
    fig_profit_factor_histogram.suptitle('Figure 1: Profit Factor For Each Timeframe', fontsize=16, y=1.01)
    fig_profit_factor_histogram.subplots_adjust(wspace=0.4, hspace=0.6)
    profit_factor_histogram_all = fig_profit_factor_histogram.add_subplot(len(timeframe_params)+1,1,1)
    profit_factor_histogram_all.hist(df.profit_factor, bins=48)
    profit_factor_histogram_all.set_title("profit factor distribution for all rows")
    profit_factor_histogram_all.set_xlabel("profit factor")
    profit_factor_histogram_all.set_ylabel("freq")
    
    profit_factor_histogram_all.set_xlim(0.7,1.4)
    
    for i, timeframe in enumerate(timeframe_params):
        profit_factor_histogram = fig_profit_factor_histogram.add_subplot(len(timeframe_params) + 1,1, 2 + i)
        profit_factor_histogram.hist(columns_picked_up.query("timeframe==" + str(timeframe)).profit_factor, bins=48)
        profit_factor_histogram.set_title("profit factor distribution for timeframe=" + str(timeframe))
        profit_factor_histogram.set_xlabel("profit factor")
        profit_factor_histogram.set_ylabel("freq")
        
        profit_factor_histogram.set_xlim(0.7,1.4)
        
    fig_profit_factor_histogram.tight_layout()
    
profit_factor_analysis(merged_results)
    

# Profit factor histogram summary
## distribution

As you see, Figure 1 hisograms show tendency that the shorter timeframe, the better profit factor.
However, it may be implied that the result caused by just a overfitting and such parameters with short timeframe
would work not well in real environment.

In this part, record the average rank for each timeframe and parameter.
The calculation order is bottom -> middle -> top, because the shorter tick has larger effect to the result.
Hence the result of calculation regard as a representative and optimal parameter for each timeframe.

After that, simulate the investment result 100,000 times using bootstrap method with these representative parameters,
then check the investment simulation result of average, mode, median and percentile.

Figure1の最初のグラフを見ると、きれいな双峰分布になっていることが見て取れる。
続いてtimeframe別のグラフを見ていくと、timeframeが小さくなっていくにつれてプロフィットファクターが大きくなっていることがわかる。
だがこの傾向があまりに明白であることと、timeframe、すなわち時間足が短くなればなるほど良い結果が得られることから、
これらのパラメータが過剰にデータセットに適合していて、本番環境ではうまくいかない可能性がある。

そこで各timeframeごとに、それぞれのパラメーターに対して平均的な順位を記録する。
結果への影響力は、短期的な価格変動の効果を受けにくい順に大きいため、bottom -> middle -> topの順で平均的な順位を計算する。
その中で得られた最適なパラメータを各タイムフレームの代表的なパラメーターとする。

これらの代表的なパラメータを用いて、各タイムフレームの代表的なパラメータごとにブートストラップ法を用いて10万回シミュレーションを行う。
これの最終的な資産額の平均値、最頻値、中央値、パーセンタイル区間を確認する。
